In [2]:
import random
import time

from settings import Settings
from script import Script
from events import Events

class Game:
    
    def __init__(self):
        
        self.settings = Settings()
        self.script = Script()
        self.events = Events(self)

        self.autogame = 0
        self.input = 0
        #自动间隔时间
        self.sleeptime = 2 
           
    def run_game(self):
        """游戏主循环"""
        print("Siege of HengShai")
        #开始界面
        self.prologue()
        while self.settings.game:
            #数值
            self.update()
            #检查输入框
            self._check_input()
            #当前主事件
            self.total_game()
            #检查是否解封
            self._check_end()
            self._check_states()
            self.debug()
            #自动游戏后加入时间间隔
            if self.autogame == 1:
                time.sleep(self.sleeptime)
        if self.settings.ending:
            self.ending()
        input("\n按任意键退出游戏。")
    
    def update(self):

        self.settings.day += 1
        self.settings.luck = random.randint(1,10)
        
        #鸡蛋
        if self.settings.beggining:
            self.settings.eggs -=2
            if self.settings.eggs <=0:
                self.settings.mood -= 1
                self.settings.eggs = 0
        #根据日期切换城市状态。
        if self.settings.day == 5:
            self.settings.mode = 1
            
        elif self.settings.day ==20:
            self.settings.mode = 2
                                    
    def total_game(self):
        
        """决定选择哪个主事件"""
        if self.settings.arc_game:
            #方舱
            self.arc_game()
        else:
            #主事件
            self.main_game()
        
    def main_game(self):
        
        """主要故事线"""
        print(f"\n【今天是封城的第{self.settings.day}天】")
        self.events.operations_events()
        self.events.important_events()
        self.events.NA_events()
        self.events.random_events()
        self.events.cyber_events()
        self.events.sleep_events()
        #self._check_hungry()
        #阳性检定
        self._check_positive()
    
    def arc_game(self):
        
        """方舱故事线"""
        self.settings.arc_day +=1
        print(f"\n【今天是封城的第{self.settings.day}天】")
        print(f"【今天是来到方舱后的第{self.settings.arc_day}天】")        
        self.events.arc_events()
        #出舱判定
        if self.settings.arc_day ==self.settings.positive_days :
            self.settings.beggining = True
            self.settings.arc_game = False
            self.settings.immune = 10
            print(self.script.negative1)
                
            
    def _check_begin(self):
        """游戏开始前数值初始化"""
        str = input("是否开始游戏？(y/n)")
        if str.upper() =='Y':
            self.settings.prologue = False
            self.settings.beggining = True
            self.settings.game = True
            self.settings.immune = self.settings.strength
            self.settings.eggs = self.settings.wealth*3+10
            print("\n当我年轻的时候，我记得任何事情，无论它是否发生过。——马克.吐温")
            
        elif str.upper() == 'N':
            self.prologue()
        else:
            print("请输入正确的选择(y/n)")
            self._check_begin()
            
    def _check_end(self):
        """检查封锁日期"""
        if self.settings.day == self.settings.days:
            self.clear_all()
            self.settings.ending =True
            
    def _check_states(self):
        
        while self.settings._check_eggs and self.settings.beggining:
            
            if self.settings.eggs <=0:
                print("\n你消耗完了所有的鸡蛋，失去了重要的蛋白质补充。\n你的免疫力下降了，你的心情也开始变得更糟。")
                self.settings.immune -=1
                self.settings._check_eggs = False
            break
            
        while self.settings._check_mood:
            if self.settings.mood <=0:
                print("\n高压下的生活让你的心情跌倒了极点。\n你的免疫力下降了。")
                self.settings.immune -=1
                self.settings._check_mood = False
            break
        if self.settings.mood <=0:
            self.settings.mood = 0
            
    def _check_input(self):
        """输入检查"""
        while self.autogame == 0:
            self.input = input("\n...任意键继续，按a自动游戏，按e退出游戏...")
            if self.input.upper() == 'A':
                self.autogame =1
            elif self.input.upper() =='E':
                self.clear_all()
                print("\n游戏终止，感谢您进行游戏。")                    

            break

    def _check_positive(self):
        """阳性检测"""
        if self.settings.immune <=0 and self.settings.day+7 <self.settings.days :
            print(self.script.positive)
            self.settings.end_arc = True
            if self.settings.luck >8:
                #会展中心。
                print(self.script.positive_luck)
            else:
                #无家可归。
                print(self.script.positive_fuck)
                input("")
                print(self.script.positive_fuck2)
                input("")
                print(self.script.positive_fuck3)
                self.settings.mood -=20
                self.settings.end_travel =True
                
            self.settings.beggining = False
            self.settings.arc_game = True
            
    def clear_all(self):
        """清理所有主循环的flag,给结尾制造条件"""
        self.settings.beggining = False
        self.settings.arc_game = False
        self.settings.game = False
    
    def _get_information(self):
        """自定义人物部分"""
        print("四项属性上限为20点。")
        try:
            self.settings.wealth = abs(int(input("请选择你的壕力")))
            self.settings.strength = abs(int(input("请选择你的体格")))
            self.settings.Ingenuity = abs(int(input("请选择你的才智")))
            self.settings.Cyber = 20 - self.settings.wealth -self.settings.strength - self.settings.Ingenuity 
        except ValueError:
            print("\n识别失败，请输入数字")
            self._get_information()
                
    def prologue(self):
        '''开始界面'''
        while self.settings.prologue:
            self._get_information()

            if self.settings.Cyber < 0 :
                print("\n超出最大分配点数！")
                continue

            print(f"\n您的壕力为{self.settings.wealth}")
            print(f"您的体格为{self.settings.strength}")
            print(f"您的才智为{self.settings.Ingenuity}")
            print(f"您的网龄为{self.settings.Cyber}\n")

            self._check_begin()
            
    def ending(self):
        """尾声和分支结局"""       
        print(self.script.end_1)
        input('')
        #print(self.settings.reputation)
        if self.settings.reputation >= 3:
            print(self.script.end_reputation)
            input('')
        
        #方舱分支
        if self.settings.end_arc == True:
            print(self.script.end_arc)
            input('')
            if self.settings.end_travel ==True:
                print(self.script.end_arc_travel)
                input('')
        else:
            print(self.script.end_arc_none)
            input('')
            
        #张麻子
        if self.settings.friendship_zmz:
            print(self.script.end_zmz)
            input('')
        
        if self.settings.mood>60:
            print(self.script.end_mood)
        else:
            print(self.script.end)
            
    def debug(self):
        #print(f"\n免疫值：{self.settings.immune},鸡蛋数目：{self.settings.eggs},心情：{self.settings.mood}\n运气：{self.settings.luck},声望：{self.settings.reputation}")
        print(f"健康：{self.settings.immune},心情：{self.settings.mood},声望：{self.settings.reputation}，城市紧张度：{self.settings.mode+1}，垃圾桶：{self.settings.default}")

if __name__ == '__main__':
    siege = Game()
    siege.run_game()

Siege of HengShai
四项属性上限为20点。
请选择你的壕力5
请选择你的体格6
请选择你的才智7

您的壕力为5
您的体格为6
您的才智为7
您的网龄为2

是否开始游戏？(y/n)y

当我年轻的时候，我记得任何事情，无论它是否发生过。——马克.吐温

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第1天】
上面要求打赢疫情防控大仗硬仗。

突然起来的封城令所有横沙人都措手不及，但你是一名死肥宅。
由于平时有囤积物资的意识，封城并没有对你造成太多的困扰。
你的同事并没有那么好运，他们当中有些人平时靠外卖度日，在封城前只准备了不到一周的干粮，他们开始对可能迎来的食品短缺感到不安。
这种焦虑很快蔓延到了整个公司，居家办公的效率自然无从谈起。

为了让自己的心态保持平和，你选择让自己与现实脱钩，再一次隐身于伟大的互联网。

但很快你发现，不同空间与不同时间的负面情绪都汇集于此，
互联网所展现出来的横沙比你自身感受到的要严峻得多。
无奈之下，你打开了Steam，这里有着上千款你买来从未玩过的游戏等待着你的幸临。

不管怎么说，你今晚的心情还算可以,
你决定多打一会游戏再睡觉。
健康：6,心情：60,声望：0，城市紧张度：1，垃圾桶：100

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第2天】
今天不需要做核酸。
居委将试剂盒分装好，送到了楼下大堂。
你被告知一天要测一次抗原,
最后要将抗原结果传到小程序上。

你注意到测核酸的队伍里有人人西装革履，有人衣着光鲜，有人油头粉面。
好像测得不是核酸，测的是小资情调。

【判定失败】：
你是一位实用主义者，你无法理解这种搔首弄姿的行为。
你穿着拖鞋大裤衩，在队伍中成为了最亮眼的仔。

回到寝室后，你打开抖音，尝试融入最新的潮流。
但你发现自己完全无法忍受其中嘈杂的特效和背景音乐。
你在这一刻意识到，你已经老了。

不管怎么说，你今晚的心情还算可以,
你决定多打一会游戏再睡觉。
健康：6,心情：60,声望：0，城市紧张度：1，垃圾桶：100

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第3天】
上面要求尽锐出战，攻坚克难。

你六点不到便被叫起来去做核酸，小区内排起了长长的队伍。
此起彼伏的喇叭播报让你萌生出了十七号城市的幻视，只不过这次你听


...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第27天】
你今天早上10点钟被叫下来测核酸。

你无意中找到了一个低价跑腿的渠道，购买了24个鸡蛋。

【判定成功】：
你想起有不少跑腿小哥为了避免困在小区里，最近一直睡在桥洞和绿化带中。
为了避免感染，你让他将鸡蛋拴在了小区护栏上。过一会再亲自去领取。

回到寝室后，你翻开了辜朝明写的《大衰退时代》，
尽管你完全看不懂上面的经济学术语
但里面所描绘的图景使你感同身受。

不管怎么说，你今晚的心情还算可以,
你决定多打一会游戏再睡觉。
健康：2,心情：62,声望：3，城市紧张度：3，垃圾桶：130

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第28天】
上面要求坚决打赢疫情防控阻击战歼灭战。

你今天早上10点钟被叫下来测核酸。
很幸运，今天只要捅喉咙。

你的邻居不慎被菜刀割到了手，血流不止。
他上门找你求助。

【判定成功】：
你注意到手指只是慢慢向外渗血，大概率是静脉受伤。
你给他采取了压迫止血，你知道用动脉止血方法的话手指可能会坏死。
邻居辗转去了四个医院也没能打上破伤风疫苗。
你在简陋的条件下，给他消毒包扎。
你的邻居对你致以最为诚挚的谢意。

回到寝室后，你打开豆瓣，翻到了一本书叫《文红章说隔离》，
这本曾经售价70元的书现在只要九块九，还包邮。

不管怎么说，你今晚的心情还算可以,
你决定多打一会游戏再睡觉。
健康：0,心情：62,声望：4，城市紧张度：3，垃圾桶：131

...任意键继续，按a自动游戏，按e退出游戏...

【今天是封城的第29天】
你今天早上9点钟被叫下来测核酸。
你感觉再这么测下去，鼻粘膜都要被捅破了。

你的邻居不慎被菜刀割到了手，血流不止。
他上门找你求助。

【判定成功】：
你注意到手指只是慢慢向外渗血，大概率是静脉受伤。
你给他采取了压迫止血，你知道用动脉止血方法的话手指可能会坏死。
邻居辗转去了四个医院也没能打上破伤风疫苗。
你在简陋的条件下，给他消毒包扎。
你的邻居对你致以最为诚挚的谢意。

回到寝室后，你打开微博，看到了仓库失火的消息。
你下意识认为这是火龙烧仓，但你没有证据。

不管怎么说，你今晚的心情还算可以,
你决定多打一会游戏再睡觉。
健康：-2,心情：62,声望：5，城市紧张度：3，垃圾桶：132

...任